In [1]:
%load_ext cython

In [2]:
import numpy as np

In [3]:
%%cython
# cython: infer_types = True
# cython: boundscheck = False
# cython: wraparound = False
cimport cython
from numpy import zeros, double as npy_double
from libc.math cimport abs, fmax

def JerkMetric(const double[:, :, :] x, double fsample):
    cdef Py_ssize_t m = x.shape[0], n = x.shape[1], p = x.shape[2], i, j, k
    cdef double amplitude, scale, jsum

    jerk_metric = zeros((m, p), dtype=npy_double)
    cdef double[:, ::1] jerk = jerk_metric

    for i in range(m):
        for k in range(p):
            jsum = 0.
            amplitude = abs(x[i, 0, k])
            for j in range(1, n):
                jsum += ((x[i, j, k] - x[i, j-1, k]) * fsample)**2
                if abs(x[i, j, k]) > amplitude:
                    amplitude = abs(x[i, j, k])
            
            scale = 360. * amplitude**2
            jerk[i, k] = jsum / (2 * scale * fsample)
    
    return jerk_metric


def DimensionlessJerk(const double[:, :, :] x, unsigned int stype):
    # stype: 1 == 'velocity', 2 == 'acceleration', 3 == 'jerk'
    cdef Py_ssize_t m = x.shape[0], n = x.shape[1], p = x.shape[2], i, j, k

    dimless_jerk = zeros((m, p), dtype=npy_double, order='C')
    cdef double[:, ::1] jerk = dimless_jerk
    cdef double amplitude, jsum

    if stype == 1:
        for i in range(m):
            for k in range(p):
                jsum = 0.
                amplitude = fmax(abs(x[i, 0, k]), abs(x[i, n-1, k]))
                for j in range(1, n-1):
                    jsum += (x[i, j+1, k] - 2 * x[i, j, k] + x[i, j-1, k])**2
                    if abs(x[i, j, k]) > amplitude:
                        amplitude = abs(x[i, j, k])

                jerk[i, k] = -(n**3 * jsum) / amplitude**2
    elif stype == 2:
        for i in range(m):
            for k in range(p):
                jsum = 0.
                amplitude = abs(x[i, 0, k])
                for j in range(1, n):
                    jsum += (x[i, j, k] - x[i, j-1, k])**2
                    if abs(x[i, j, k]) > amplitude:
                        amplitude = abs(x[i, j, k])

                jerk[i, k] = -(n * jsum) / amplitude**2
    elif stype == 3:
        for i in range(m):
            for k in range(p):
                jsum = 0.
                amplitude = 0.
                for j in range(n):
                    jsum += x[i, j, k]**2
                    if abs(x[i, j, k]) > amplitude:
                        amplitude = abs(x[i, j, k])

                jerk[i, k] = -jsum / (n * amplitude**2)
    return dimless_jerk

In [4]:
from jerk import jerkmetric as fjerk, dimensionlessjerkmetric as fdjerk

In [5]:
x = np.random.rand(50000, 150, 3)
xf = np.asfortranarray(x.transpose([1, 2, 0]))

In [7]:
%timeit JerkMetric(x, 50.0)

27.3 ms ± 3.08 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [8]:
%timeit fjerk(xf, 50.0)

20.1 ms ± 19.2 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [9]:
np.allclose(fjerk(xf, 50.0).T, JerkMetric(x, 50.0))

True

In [10]:
%timeit DimensionlessJerk(x, 1)
%timeit DimensionlessJerk(x, 2)
%timeit DimensionlessJerk(x, 3)

42.4 ms ± 18.1 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)
24.7 ms ± 27 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)
24.4 ms ± 18.5 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [11]:
%timeit fdjerk(xf, 1)
%timeit fdjerk(xf, 2)
%timeit fdjerk(xf, 3)

30.4 ms ± 4.72 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)
21.6 ms ± 20.4 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)
20.2 ms ± 38.6 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [6]:
print(np.allclose(fdjerk(xf, 1).T, DimensionlessJerk(x, 1)))
print(np.allclose(fdjerk(xf, 2).T, DimensionlessJerk(x, 2)))
print(np.allclose(fdjerk(xf, 3).T, DimensionlessJerk(x, 3)))

True
True
True
